In [5]:
#import the necessary modules
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from numpy  import array
from random import randrange
from imutils import paths
import timeit
#load the image
#img_original = cv2.imread('/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid1HQ.jpg')
#plt.imshow(img_original)
#function to convert the cell (string) to list of coordinates (x,y) in order to
#be easier to extract the coordinates later
def Convert(string): 
    li = list(string.split(",")) 
    return li 

In [6]:
nests_folder="/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images"
imagePathsnests = sorted(list(paths.list_images(nests_folder)))
imagePathsnests

['/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid3MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5MQ.jpg']

In [7]:
#imagePathsnests=imagePathsnests[2:10]

In [8]:
imagePathsnests

['/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid3MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid4MQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5HQ.jpg',
 '/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/Nid5MQ.jpg']

In [ ]:
#opencv opens the image with the wrong colour channels
#this code corrects that
start = timeit.default_timer()
for image_path in imagePathsnests:
    img_original=cv2.imread(image_path)
    img=cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
    #loop
    #load of the manually annotated polygons using VGG annotator
    df_original= pd.read_csv("/mnt/ubuntu_disk/Ubuntu_drive/Nests/Images/FiveNestAnnotedTest2.csv")
    #select only the current image
    df=df_original[df_original["filename"]==image_path.split("/")[-1:][0]]
    #print dataframe
    #print(df)
    #loop through all lines of the df to get the coordinates of all polygons for same image
    #create a list to store all xx and all yy of all polygons
    all_coordx=[]
    all_coordy=[]
    #reset the index of the line so that the next loop doesn't give error
    df=df.reset_index()
    for d in range(0, len(df)):
    #remove the text that is not needed and keep only the coordinates
        coord=df["region_shape_attributes"][d].replace('{"name":"polygon","all_points_x":', "")
        coord=coord.replace(',"all_points_y":', "")
        coord=coord.replace(']', "")
        coord=coord.replace('}', "")
        #create a list for the xx coordinates and another for the yy coordinates
        coordx=coord.split("[")[1]
        coordx=Convert(coordx)
        coordy=coord.split("[")[2]
        coordy=Convert(coordy)
        all_coordx.append(coordx)
        all_coordy.append(coordy)
    window=10
    square_index=1
    #double loop to go through all polygons and within each polygon create a list with all points
    for p in range(0,len(all_coordx)):
        #create a final list by merging the xx and yy lists
        polycoordinates=[]
        coordx=all_coordx[p]
        coordy=all_coordy[p]
        for i in range(len(coordx)):
            polycoordinates.append([coordx[i],coordy[i]])
            #convert to array in order to extract the background later
        polycoordinates = array(polycoordinates, np.int32)
        #Crop the bounding box of the polygon
        rect = cv2.boundingRect(polycoordinates)
        x,y,w,h = rect
        croped = img_original[y:y+h, x:x+w].copy()
        # create a mask corresponding to the area of the selected polygon
        polycoordinates = polycoordinates - polycoordinates.min(axis=0)
        mask = np.zeros(croped.shape[:2], np.uint8)
        cv2.drawContours(mask, [polycoordinates], -1, (255, 255, 255), -1, cv2.LINE_AA)
        #remove the background
        dst = cv2.bitwise_and(croped, croped, mask=mask)
        #re-convert the polygon array to list again in order to
        #use the functions from the shapely module. this module includes a function
        #that allows to check if a point is inside a polygon, using functions "Point(x,y)"
        #and "Polygon(list_of_coordinates)"
        polygon_x=[]
        for j in range(0,len(polycoordinates)):
            polygon_x.append((polycoordinates[j][0],polycoordinates[j][1]))
        #convert to polygon object type of shapely module
        poly_final=Polygon(polygon_x)
        #this loop makes random squares and
        #keeps the ones that are inside the region of finterest
        #create a copy of the image for ilustration
        dst_test=dst.copy()
        

        #get maximum length
        max_size=max(dst.shape)
        r=[0.2,0.25,0.3,0.35,0.4,0.45,0.5]
        for ri in r:
            size=max_size*ri
            #loop until it reaches the number of squares needed
            #create a variable to save the yy point
            #current_yy=0
            #number of squares needed
            attempts=0
            #create a list to store the squares
            squares=[]
            num_squares=50
            print(ri)

            #loop until it reaches the number of squares needed
            while len(squares)<num_squares:
                if attempts>2000:
                    break
                print(attempts)
                #randomly chose a x and a y coordinates
                current_xx=randrange(0, dst.shape[0])
                current_yy=randrange(0, dst.shape[1])
                #randomly chose the length of the edges of the square (in this case between 10 to 50)
                #get all points inside the randomly determined square
                X, Y = np.mgrid[current_xx:current_xx+size, current_yy:current_yy+size]
                points=np.vstack((X.ravel(), Y.ravel()))
                #variable to note if all points are inside the polygon of interest
                all_true=1
                attempts=attempts+1
                #loop through all points and if any of the points is outside of the 
                #region of interest change "all_true" variable to 0
                for ii in range(0, len(points[0])):
                    if poly_final.contains((Point(points[0][ii], points[1][ii]))) == False:
                        all_true=0
                        break
                #only if all_true=1 include that square in the list
                if all_true==1:
                    squares.append([(round(current_xx),round(current_yy)),(round(current_xx+size),round(current_yy+size))])
                    cropped=dst_test[round(current_yy):round(current_yy)+round(size),round(current_xx):round(current_xx)+round(size)]
                    #export cropped image to a folder
                    print(len(squares))
                    filename="/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/"
                    filename=filename+image_path.split("/")[-1:][0].split(".")[0]+"_"+"Polygon"+str(p+1)+"_"+str(ri)+"_"+str(square_index)+".jpg"
                    print(filename)
                    square_index=square_index+1
                    cv2.imwrite(filename, cropped)
            finaltest=dst.copy()
            for i in range(0, len(squares)):
                finaltest=cv2.rectangle(finaltest,squares[i][0], squares[i][1],(0,255,0),3)
            filename_final="/mnt/ubuntu_disk/Ubuntu_drive/Nests/Results/"
            filename_final=filename_final+image_path.split("/")[-1:][0].split(".")[0]+"_"+"Polygon"+str(p+1)+"_"+str(ri)+"_"+"Final"+".jpg"
            cv2.imwrite(filename_final, finaltest)
stop = timeit.default_timer()

print('Time: ', stop - start)